In [1]:
import time
import subprocess
import matplotlib.pyplot as plt
import psutil
import os
import re
import threading
import csv
import signal
import numpy as np
import math

In [2]:
programs = ["/Users/apple/gForthVM/DT_generator","/Users/apple/gForthVM/IDT_generator","/Users/apple/gForthVM/subroutine_generator"]

In [3]:
directory = './grammars'
files = os.listdir(directory)
print(files)

['math.json', 'recursive.json', 'simple_grammar.json', 'css.json', 'html.json']


In [4]:
depth = [1,2,4,8,16,32,64,128,256,512,1024,2048]

In [6]:
throughput_rates = []

def monitor_process(process, timeout):
    time.sleep(timeout)
    if process.poll() is None:  
        process.terminate()
        try:
            process.wait(timeout=5)
        except subprocess.TimeoutExpired:
            process.kill()
            print("Process was killed after timeout")

def read_output(process):
    global throughput_rates
    while True:
        output = process.stdout.readline()
        if process.poll() is not None and output == '':
            break
        if output:
            match = re.search(r'^(\d+\.\d+) KB/s$', output)  # Modified regular expression pattern
            if match:
                throughput_rate = float(match.group(1))
                throughput_rates.append(throughput_rate)


result = {}
timeout = 10

for i in range(len(programs)):
    program_name = programs[i]
    result[program_name] = {}
    for j in range(len(files)):
        file_name = files[j]
        result[program_name][file_name] = {}
        for k in range(len(depth)):
            depth_value = depth[k]
            throughput_rates = []
            process = subprocess.Popen(
                [program_name, "-depth", str(depth[k]), "-path", f"./grammars/{files[j]}", "-o", f"{file_name}.fth", "--show"],
                stderr=subprocess.PIPE,
                stdout=subprocess.PIPE,
                text=True,
            )
            monitor_thread = threading.Thread(target=monitor_process, args=(process, timeout))
            monitor_thread.daemon = True  
            monitor_thread.start()
            read_output(process)
            monitor_thread.join()
            if process.poll() is None:
                process.terminate()
                try:
                    process.wait(timeout=5)
                except subprocess.TimeoutExpired:
                    process.kill()
            if throughput_rates:
                avg_throughput_rate = sum(throughput_rates) / len(throughput_rates)
            else:
                avg_throughput_rate = 0
            result[program_name][file_name][depth_value] = avg_throughput_rate
            print(f"{program_name} {file_name} {depth_value} {avg_throughput_rate}")
print(result)


/Users/apple/gForthVM/DT_generator math.json 1 62946.40337372072
/Users/apple/gForthVM/DT_generator math.json 2 51701.2419632682
/Users/apple/gForthVM/DT_generator math.json 4 22664.506088615486
/Users/apple/gForthVM/DT_generator math.json 8 17.725255423416122
/Users/apple/gForthVM/DT_generator math.json 16 4.345733746036393
/Users/apple/gForthVM/DT_generator math.json 32 2.836633588106156


KeyboardInterrupt: 